In [1]:
import pandas as pd

pd.set_option('display.max_columns', 500)

In [16]:
from team_map import map_team_names

In [1]:
total_data = pd.read_csv('../combine/FinalJoinedData.csv', index_col=0)

NameError: name 'pd' is not defined

In [18]:
cols_unnamed = []
for col in total_data.columns:
    if 'Unnamed' in col:
        cols_unnamed.append(col)
        
total_data = total_data.drop(columns=cols_unnamed)

In [19]:
total_data.head()

,Seed,Team,Conference,AdjEM,AdjO,AdjD,AdjT,Luck,Opp AdjEM,OppO,OppD,NCSOS AdjEM,W,L,ast_pct,blk_pct,efg_pct,fg3a_per_fga_pct,ft_rate,fta_per_fga_pct,losses_conf,losses_home,losses_visitor,opp_pts,pts,sos,srs,tov_pct,trb_pct,ts_pct,wins_conf,wins_home,wins_visitor,def_rtg,opp_fta_per_fga_pct,opp_fg3a_per_fga_pct,opp_ts_pct,opp_trb_pct,opp_ast_pct,opp_stl_pct,opp_blk_pct,opp_efg_pct,opp_tov_pct,opp_orb_pct,opp_ft_rate,num_FR,num_SO,num_JR,num_F,num_G,num_C,ast_per_g_1,ast_per_g_2,ast_per_g_3,ast_per_g_4,ast_per_g_5,ast_per_g_6,ast_per_g_7,ast_per_g_8,ast_per_g_9,ast_per_g_10,ast_per_g_11,ast_per_g_12,ast_per_g_13,ast_per_g_14,blk_per_g_1,blk_per_g_2,blk_per_g_3,blk_per_g_4,blk_per_g_5,blk_per_g_6,blk_per_g_7,blk_per_g_8,blk_per_g_9,blk_per_g_10,blk_per_g_11,blk_per_g_12,blk_per_g_13,blk_per_g_14,drb_per_g_1,drb_per_g_2,drb_per_g_3,drb_per_g_4,drb_per_g_5,drb_per_g_6,drb_per_g_7,drb_per_g_8,drb_per_g_9,drb_per_g_10,drb_per_g_11,drb_per_g_12,drb_per_g_13,drb_per_g_14,fg2_pct_1,fg2_pct_2,fg2_pct_3,fg2_pct_4,fg2_pct_5,fg2_pct_6,fg2_pct_7,fg2_pct_8,fg2_pct_9,fg2_pct_10,fg2_pct_11,fg2_pct_12,fg2_pct_13,fg2_pct_14,fg2_per_g_1,fg2_per_g_2,fg2_per_g_3,fg2_per_g_4,fg2_per_g_5,fg2_per_g_6,fg2_per_g_7,fg2_per_g_8,fg2_per_g_9,fg2_per_g_10,fg2_per_g_11,fg2_per_g_12,fg2_per_g_13,fg2_per_g_14,fg2a_per_g_1,fg2a_per_g_2,fg2a_per_g_3,fg2a_per_g_4,fg2a_per_g_5,fg2a_per_g_6,fg2a_per_g_7,fg2a_per_g_8,fg2a_per_g_9,fg2a_per_g_10,fg2a_per_g_11,fg2a_per_g_12,fg2a_per_g_13,fg2a_per_g_14,fg3_pct_1,fg3_pct_2,fg3_pct_3,fg3_pct_4,fg3_pct_5,fg3_pct_6,fg3_pct_7,fg3_pct_8,fg3_pct_9,fg3_pct_10,fg3_pct_11,fg3_pct_12,fg3_pct_13,fg3_pct_14,fg3_per_g_1,fg3_per_g_2,fg3_per_g_3,fg3_per_g_4,fg3_per_g_5,fg3_per_g_6,fg3_per_g_7,fg3_per_g_8,fg3_per_g_9,fg3_per_g_10,fg3_per_g_11,fg3_per_g_12,fg3_per_g_13,fg3_per_g_14,fg3a_per_g_1,fg3a_per_g_2,fg3a_per_g_3,fg3a_per_g_4,fg3a_per_g_5,fg3a_per_g_6,fg3a_per_g_7,fg3a_per_g_8,fg3a_per_g_9,fg3a_per_g_10,fg3a_per_g_11,fg3a_per_g_12,fg3a_per_g_13,fg3a_per_g_14,fg_pct_1,fg_pct_2,fg_pct_3,fg_pct_4,fg_pct_5,fg_pct_6,fg_pct_7,fg_pct_8,fg_pct_9,fg_pct_10,fg_pct_11,fg_pct_12,fg_pct_13,fg_pct_14,fg_per_g_1,fg_per_g_2,fg_per_g_3,fg_per_g_4,fg_per_g_5,fg_per_g_6,fg_per_g_7,fg_per_g_8,fg_per_g_9,fg_per_g_10,fg_per_g_11,fg_per_g_12,fg_per_g_13,fg_per_g_14,fga_per_g_1,fga_per_g_2,fga_per_g_3,fga_per_g_4,fga_per_g_5,fga_per_g_6,fga_per_g_7,fga_per_g_8,fga_per_g_9,fga_per_g_10,fga_per_g_11,fga_per_g_12,fga_per_g_13,fga_per_g_14,ft_pct_1,ft_pct_2,ft_pct_3,ft_pct_4,ft_pct_5,ft_pct_6,ft_pct_7,ft_pct_8,ft_pct_9,ft_pct_10,ft_pct_11,ft_pct_12,ft_pct_13,ft_pct_14,ft_per_g_1,ft_per_g_2,ft_per_g_3,ft_per_g_4,ft_per_g_5,ft_per_g_6,ft_per_g_7,ft_per_g_8,ft_per_g_9,ft_per_g_10,ft_per_g_11,ft_per_g_12,ft_per_g_13,ft_per_g_14,fta_per_g_1,fta_per_g_2,fta_per_g_3,fta_per_g_4,fta_per_g_5,fta_per_g_6,fta_per_g_7,fta_per_g_8,fta_per_g_9,fta_per_g_10,fta_per_g_11,fta_per_g_12,fta_per_g_13,fta_per_g_14,height_1,height_2,height_3,height_4,height_5,height_6,height_7,height_8,height_9,height_10,height_11,height_12,height_13,height_14,mp_per_g_1,mp_per_g_2,mp_per_g_3,mp_per_g_4,mp_per_g_5,mp_per_g_6,mp_per_g_7,mp_per_g_8,mp_per_g_9,mp_per_g_10,mp_per_g_11,mp_per_g_12,mp_per_g_13,mp_per_g_14,orb_per_g_1,orb_per_g_2,orb_per_g_3,orb_per_g_4,orb_per_g_5,orb_per_g_6,orb_per_g_7,orb_per_g_8,orb_per_g_9,orb_per_g_10,orb_per_g_11,orb_per_g_12,orb_per_g_13,orb_per_g_14,pf_per_g_1,pf_per_g_2,pf_per_g_3,pf_per_g_4,pf_per_g_5,pf_per_g_6,pf_per_g_7,pf_per_g_8,pf_per_g_9,pf_per_g_10,pf_per_g_11,pf_per_g_12,pf_per_g_13,pf_per_g_14,pts_per_g_1,pts_per_g_2,pts_per_g_3,pts_per_g_4,pts_per_g_5,pts_per_g_6,pts_per_g_7,pts_per_g_8,pts_per_g_9,pts_per_g_10,pts_per_g_11,pts_per_g_12,pts_per_g_13,pts_per_g_14,stl_per_g_1,stl_per_g_2,stl_per_g_3,stl_per_g_4,stl_per_g_5,stl_per_g_6,stl_per_g_7,stl_per_g_8,stl_per_g_9,stl_per_g_10,stl_per_g_11,stl_per_g_12,stl_per_g_13,stl_per_g_14,tov_per_g_1,tov_per_g_2,tov_per_g_3,tov_per_g_4,tov_per_g_5,tov_per_g_6,tov_per_g_7,tov_per_g_8,tov_per_g_9,tov_per_g_

In [20]:
results = pd.read_csv('../gamesdata/Results.csv', index_col=0)

results['winner'] = results['winner'].apply(map_team_names)
results['team_one'] = results['team_one'].apply(map_team_names)
results['team_two'] = results['team_two'].apply(map_team_names)

In [21]:
results.head()

,year,winner,team_one,team_two,score_one,score_two
0,2002,Maryland,Maryland,Siena,85,70
1,2002,Maryland,Maryland,Wisconsin,87,57
2,2002,Maryland,Maryland,Kentucky,78,68
3,2002,Maryland,Maryland,Connecticut,90,82
4,2002,Maryland,Maryland,Kansas,97,88


In [22]:
print (results.shape)

(1112, 6)


In [23]:
data_teams = set(total_data['Team'].unique())
results_teams = set(results['team_one'].unique()) 
results_teams.update(set(results['team_two'].unique()))

print (sorted(data_teams - results_teams))
print (sorted(results_teams - data_teams))

['Abilene Christian', 'Appalachian State', 'Arkansas State', 'Army', 'Ball State', 'Bethune Cookman', 'Bowling Green', 'Brown', 'Bryant', 'Cal Baptist', 'Campbell', 'Canisius', 'Central Arkansas', 'Charleston Southern', 'Chicago State', 'Colgate', 'Columbia', 'Dartmouth', 'Denver', 'Drexel', 'Duquesne', 'East Carolina', 'Eastern Illinois', 'Eastern Michigan', 'Elon', 'Evansville', 'FIU', 'Fairfield', 'Fordham', 'Furman', 'Gardner Webb', 'Georgia Southern', 'Grambling', 'Grand Canyon', 'Hartford', 'High Point', 'Hofstra', 'Houston Baptist', 'Howard', 'Idaho', 'Idaho State', 'Illinois State', 'Incarnate Word', 'Jacksonville', 'Kennesaw State', 'Longwood', 'Louisiana Monroe', 'Louisiana Tech', 'Loyola Marymount', 'Maine', 'Marist', 'Maryland Eastern Shore', 'Merrimack', 'Missouri State', 'Montana State', 'NJIT', 'Navy', 'New Hampshire', 'Nicholls State', 'North Alabama', 'Northern Arizona', 'Northern Illinois', 'Omaha', 'Portland', 'Prairie View A&M', 'Presbyterian', 'Purdue Fort Wayne', 

In [26]:
def make_predictions(invert=False):
    res = []
    for idx, row in results.iterrows():
        # find the team in our total_data
        curr_year = total_data[total_data['year'] == row['year']]
#         print (f'{row["team_one"]} - {row["team_two"]}')
        curr_team_one = dict(curr_year[curr_year['Team'] == row['team_one']].iloc[0,:])
        curr_team_two = dict(curr_year[curr_year['Team'] == row['team_two']].iloc[0,:])

        result = 1 if row['winner'] == row['team_one'] else 0
        score_one = row['score_one']
        score_two = row['score_two']
        
        curr_row = dict()
        for k, v in curr_team_one.items():
            curr_row[k + '_team_one'] = v
        for k, v in curr_team_two.items():
            curr_row[k + '_team_two'] = v
        curr_row['result'] = result

        curr_row['score_one'] = score_one
        curr_row['score_two'] = score_two
        
        res.append(curr_row)
        
        if invert:
            # experiment with inverting to make more synthetic data
            result = 0 if row['winner'] == row['team_one'] else 1
            curr_row = dict()
            for k,v in curr_team_two.items():
                curr_row[k + '_team_one'] = v
            for k, v in curr_team_one.items():
                curr_row[k + '_team_two'] = v
            curr_row['result'] = result
            curr_row['score_one'] = score_two
            curr_row['score_two'] = score_one

            res.append(curr_row)
    return res

In [27]:
prediction_data = make_predictions()
prediction_invert = make_predictions(True)

In [28]:
df = pd.DataFrame(prediction_data)
df_invert = pd.DataFrame(prediction_invert)

In [29]:
df.head()

,Seed_team_one,Team_team_one,Conference_team_one,AdjEM_team_one,AdjO_team_one,AdjD_team_one,AdjT_team_one,Luck_team_one,Opp AdjEM_team_one,OppO_team_one,OppD_team_one,NCSOS AdjEM_team_one,W_team_one,L_team_one,ast_pct_team_one,blk_pct_team_one,efg_pct_team_one,fg3a_per_fga_pct_team_one,ft_rate_team_one,fta_per_fga_pct_team_one,losses_conf_team_one,losses_home_team_one,losses_visitor_team_one,opp_pts_team_one,pts_team_one,sos_team_one,srs_team_one,tov_pct_team_one,trb_pct_team_one,ts_pct_team_one,wins_conf_team_one,wins_home_team_one,wins_visitor_team_one,def_rtg_team_one,opp_fta_per_fga_pct_team_one,opp_fg3a_per_fga_pct_team_one,opp_ts_pct_team_one,opp_trb_pct_team_one,opp_ast_pct_team_one,opp_stl_pct_team_one,opp_blk_pct_team_one,opp_efg_pct_team_one,opp_tov_pct_team_one,opp_orb_pct_team_one,opp_ft_rate_team_one,num_FR_team_one,num_SO_team_one,num_JR_team_one,num_F_team_one,num_G_team_one,num_C_team_one,ast_per_g_1_team_one,ast_per_g_2_team_one,ast_per_g_3_team_one,ast_per_g_4_team_one,ast_per_g_5_team_one,ast_per_g_6_team_one,ast_per_g_7_team_one,ast_per_g_8_team_one,ast_per_g_9_team_one,ast_per_g_10_team_one,ast_per_g_11_team_one,ast_per_g_12_team_one,ast_per_g_13_team_one,ast_per_g_14_team_one,blk_per_g_1_team_one,blk_per_g_2_team_one,blk_per_g_3_team_one,blk_per_g_4_team_one,blk_per_g_5_team_one,blk_per_g_6_team_one,blk_per_g_7_team_one,blk_per_g_8_team_one,blk_per_g_9_team_one,blk_per_g_10_team_one,blk_per_g_11_team_one,blk_per_g_12_team_one,blk_per_g_13_team_one,blk_per_g_14_team_one,drb_per_g_1_team_one,drb_per_g_2_team_one,drb_per_g_3_team_one,drb_per_g_4_team_one,drb_per_g_5_team_one,drb_per_g_6_team_one,drb_per_g_7_team_one,drb_per_g_8_team_one,drb_per_g_9_team_one,drb_per_g_10_team_one,drb_per_g_11_team_one,drb_per_g_12_team_one,drb_per_g_13_team_one,drb_per_g_14_team_one,fg2_pct_1_team_one,fg2_pct_2_team_one,fg2_pct_3_team_one,fg2_pct_4_team_one,fg2_pct_5_team_one,fg2_pct_6_team_one,fg2_pct_7_team_one,fg2_pct_8_team_one,fg2_pct_9_team_one,fg2_pct_10_team_one,fg2_pct_11_team_one,fg2_pct_12_team_one,fg2_pct_13_team_one,fg2_pct_14_team_one,fg2_per_g_1_team_one,fg2_per_g_2_team_one,fg2_per_g_3_team_one,fg2_per_g_4_team_one,fg2_per_g_5_team_one,fg2_per_g_6_team_one,fg2_per_g_7_team_one,fg2_per_g_8_team_one,fg2_per_g_9_team_one,fg2_per_g_10_team_one,fg2_per_g_11_team_one,fg2_per_g_12_team_one,fg2_per_g_13_team_one,fg2_per_g_14_team_one,fg2a_per_g_1_team_one,fg2a_per_g_2_team_one,fg2a_per_g_3_team_one,fg2a_per_g_4_team_one,fg2a_per_g_5_team_one,fg2a_per_g_6_team_one,fg2a_per_g_7_team_one,fg2a_per_g_8_team_one,fg2a_per_g_9_team_one,fg2a_per_g_10_team_one,fg2a_per_g_11_team_one,fg2a_per_g_12_team_one,fg2a_per_g_13_team_one,fg2a_per_g_14_team_one,fg3_pct_1_team_one,fg3_pct_2_team_one,fg3_pct_3_team_one,fg3_pct_4_team_one,fg3_pct_5_team_one,fg3_pct_6_team_one,fg3_pct_7_team_one,fg3_pct_8_team_one,fg3_pct_9_team_one,fg3_pct_10_team_one,fg3_pct_11_team_one,fg3_pct_12_team_one,fg3_pct_13_team_one,fg3_pct_14_team_one,fg3_per_g_1_team_one,fg3_per_g_2_team_one,fg3_per_g_3_team_one,fg3_per_g_4_team_one,fg3_per_g_5_team_one,fg3_per_g_6_team_one,fg3_per_g_7_team_one,fg3_per_g_8_team_one,fg3_per_g_9_team_one,fg3_per_g_10_team_one,fg3_per_g_11_team_one,fg3_per_g_12_team_one,fg3_per_g_13_team_one,fg3_per_g_14_team_one,fg3a_per_g_1_team_one,fg3a_per_g_2_team_one,fg3a_per_g_3_team_one,fg3a_per_g_4_team_one,fg3a_per_g_5_team_one,fg3a_per_g_6_team_one,fg3a_per_g_7_team_one,fg3a_per_g_8_team_one,fg3a_per_g_9_team_one,fg3a_per_g_10_team_one,fg3a_per_g_11_team_one,fg3a_per_g_12_team_one,fg3a_per_g_13_team_one,fg3a_per_g_14_team_one,fg_pct_1_team_one,fg_pct_2_team_one,fg_pct_3_team_one,fg_pct_4_team_one,fg_pct_5_team_one,fg_pct_6_team_one,fg_pct_7_team_one,fg_pct_8_team_one,fg_pct_9_team_one,fg_pct_10_team_one,fg_pct_11_team_one,fg_pct_12_team_one,fg_pct_13_team_one,fg_pct_14_team_one,fg_per_g_1_team_one,fg_per_g_2_team_one,fg_per_g_3_team_one,fg_per_g_4_team_one,fg_per_g_5_team_one,fg_per_g_6_team_one,fg_per_g_7_team_one,fg_per_g_8_team_one,fg

In [30]:
df_invert.head()

,Seed_team_one,Team_team_one,Conference_team_one,AdjEM_team_one,AdjO_team_one,AdjD_team_one,AdjT_team_one,Luck_team_one,Opp AdjEM_team_one,OppO_team_one,OppD_team_one,NCSOS AdjEM_team_one,W_team_one,L_team_one,ast_pct_team_one,blk_pct_team_one,efg_pct_team_one,fg3a_per_fga_pct_team_one,ft_rate_team_one,fta_per_fga_pct_team_one,losses_conf_team_one,losses_home_team_one,losses_visitor_team_one,opp_pts_team_one,pts_team_one,sos_team_one,srs_team_one,tov_pct_team_one,trb_pct_team_one,ts_pct_team_one,wins_conf_team_one,wins_home_team_one,wins_visitor_team_one,def_rtg_team_one,opp_fta_per_fga_pct_team_one,opp_fg3a_per_fga_pct_team_one,opp_ts_pct_team_one,opp_trb_pct_team_one,opp_ast_pct_team_one,opp_stl_pct_team_one,opp_blk_pct_team_one,opp_efg_pct_team_one,opp_tov_pct_team_one,opp_orb_pct_team_one,opp_ft_rate_team_one,num_FR_team_one,num_SO_team_one,num_JR_team_one,num_F_team_one,num_G_team_one,num_C_team_one,ast_per_g_1_team_one,ast_per_g_2_team_one,ast_per_g_3_team_one,ast_per_g_4_team_one,ast_per_g_5_team_one,ast_per_g_6_team_one,ast_per_g_7_team_one,ast_per_g_8_team_one,ast_per_g_9_team_one,ast_per_g_10_team_one,ast_per_g_11_team_one,ast_per_g_12_team_one,ast_per_g_13_team_one,ast_per_g_14_team_one,blk_per_g_1_team_one,blk_per_g_2_team_one,blk_per_g_3_team_one,blk_per_g_4_team_one,blk_per_g_5_team_one,blk_per_g_6_team_one,blk_per_g_7_team_one,blk_per_g_8_team_one,blk_per_g_9_team_one,blk_per_g_10_team_one,blk_per_g_11_team_one,blk_per_g_12_team_one,blk_per_g_13_team_one,blk_per_g_14_team_one,drb_per_g_1_team_one,drb_per_g_2_team_one,drb_per_g_3_team_one,drb_per_g_4_team_one,drb_per_g_5_team_one,drb_per_g_6_team_one,drb_per_g_7_team_one,drb_per_g_8_team_one,drb_per_g_9_team_one,drb_per_g_10_team_one,drb_per_g_11_team_one,drb_per_g_12_team_one,drb_per_g_13_team_one,drb_per_g_14_team_one,fg2_pct_1_team_one,fg2_pct_2_team_one,fg2_pct_3_team_one,fg2_pct_4_team_one,fg2_pct_5_team_one,fg2_pct_6_team_one,fg2_pct_7_team_one,fg2_pct_8_team_one,fg2_pct_9_team_one,fg2_pct_10_team_one,fg2_pct_11_team_one,fg2_pct_12_team_one,fg2_pct_13_team_one,fg2_pct_14_team_one,fg2_per_g_1_team_one,fg2_per_g_2_team_one,fg2_per_g_3_team_one,fg2_per_g_4_team_one,fg2_per_g_5_team_one,fg2_per_g_6_team_one,fg2_per_g_7_team_one,fg2_per_g_8_team_one,fg2_per_g_9_team_one,fg2_per_g_10_team_one,fg2_per_g_11_team_one,fg2_per_g_12_team_one,fg2_per_g_13_team_one,fg2_per_g_14_team_one,fg2a_per_g_1_team_one,fg2a_per_g_2_team_one,fg2a_per_g_3_team_one,fg2a_per_g_4_team_one,fg2a_per_g_5_team_one,fg2a_per_g_6_team_one,fg2a_per_g_7_team_one,fg2a_per_g_8_team_one,fg2a_per_g_9_team_one,fg2a_per_g_10_team_one,fg2a_per_g_11_team_one,fg2a_per_g_12_team_one,fg2a_per_g_13_team_one,fg2a_per_g_14_team_one,fg3_pct_1_team_one,fg3_pct_2_team_one,fg3_pct_3_team_one,fg3_pct_4_team_one,fg3_pct_5_team_one,fg3_pct_6_team_one,fg3_pct_7_team_one,fg3_pct_8_team_one,fg3_pct_9_team_one,fg3_pct_10_team_one,fg3_pct_11_team_one,fg3_pct_12_team_one,fg3_pct_13_team_one,fg3_pct_14_team_one,fg3_per_g_1_team_one,fg3_per_g_2_team_one,fg3_per_g_3_team_one,fg3_per_g_4_team_one,fg3_per_g_5_team_one,fg3_per_g_6_team_one,fg3_per_g_7_team_one,fg3_per_g_8_team_one,fg3_per_g_9_team_one,fg3_per_g_10_team_one,fg3_per_g_11_team_one,fg3_per_g_12_team_one,fg3_per_g_13_team_one,fg3_per_g_14_team_one,fg3a_per_g_1_team_one,fg3a_per_g_2_team_one,fg3a_per_g_3_team_one,fg3a_per_g_4_team_one,fg3a_per_g_5_team_one,fg3a_per_g_6_team_one,fg3a_per_g_7_team_one,fg3a_per_g_8_team_one,fg3a_per_g_9_team_one,fg3a_per_g_10_team_one,fg3a_per_g_11_team_one,fg3a_per_g_12_team_one,fg3a_per_g_13_team_one,fg3a_per_g_14_team_one,fg_pct_1_team_one,fg_pct_2_team_one,fg_pct_3_team_one,fg_pct_4_team_one,fg_pct_5_team_one,fg_pct_6_team_one,fg_pct_7_team_one,fg_pct_8_team_one,fg_pct_9_team_one,fg_pct_10_team_one,fg_pct_11_team_one,fg_pct_12_team_one,fg_pct_13_team_one,fg_pct_14_team_one,fg_per_g_1_team_one,fg_per_g_2_team_one,fg_per_g_3_team_one,fg_per_g_4_team_one,fg_per_g_5_team_one,fg_per_g_6_team_one,fg_per_g_7_team_one,fg_per_g_8_team_one,fg

In [31]:
df.to_csv('PredictionData.csv', index=False)

In [32]:
df_invert.to_csv('PredictionDataInvert.csv', index=False)